# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise, Jack Demeter
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [2]:
## Data Pre-Processing
#### We load in individual csv files collected from Rosbag on TurtleBot3 with ROS.

In [3]:
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

savePath = "Data-32Series-Delta30-Squared.csv"

In [4]:
""" 
For each data file we:
adjust the index, 
interpolate NaN values,
drop remaining NaN values, 
drop some empty columns.

We then augment the feature space with delta columns, 
and with polynomial columns,
and label the columns with terrain, speed, and trial number
"""
for i in range(len(dataFiles)):
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    #print(df.isnull().sum().sum())
    df = df.interpolate(method='polynomial', order=1)
    #print(df.isnull().sum().sum())

    df = df.dropna()
    #print(df.isnull().sum().sum())
    df = df.reset_index().drop(columns=['Seq'])

    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])
    df['OdomPosXY'] = np.sqrt(df.OdomPosX**2 + df.OdomPosY**2)
    df = df.drop(columns=['OdomPosY', 'OdomPosX'])
    order=2
    
    dList = range(1, 302, 10)
    # dList = [2,4,8,16,32,64]
    for col in df.columns.tolist():
        if col!='Sensor':
            for d in dList:
                df[col+'Delta{}'.format(d)] = df[col].diff(d)
                if order>1:
                    for p in range(2, order+1):
                        df[col+'Delta{}Exp{}'.format(d, p)] = df[col+'Delta{}'.format(d)]**p
                        #print('Added ' + col +'Delta{}'.format(d))
        else:
            pass

    df = df.iloc[max(dList):].reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
    print("Added {} of size {} to mainDf. ".format(dataFiles[i], df.shape))
    print("Data series completed: {}/{}".format(i+1,len(dataFiles)))
    print("MainDf is now size {}".format(mainDf.shape))
    print('\n')


Added gMitTile_s15_t8.csv of size (3530, 1010) to mainDf. 
Data series completed: 1/32
MainDf is now size (3530, 1010)


Added gMitTile_s15_t9.csv of size (4030, 1010) to mainDf. 
Data series completed: 2/32
MainDf is now size (7560, 1010)


Added gMitTile_s15_t10.csv of size (4552, 1010) to mainDf. 
Data series completed: 3/32
MainDf is now size (12112, 1010)


Added gTurf_s15_t3.csv of size (5800, 1010) to mainDf. 
Data series completed: 4/32
MainDf is now size (17912, 1010)


Added gTurf_s15_t4.csv of size (5539, 1010) to mainDf. 
Data series completed: 5/32
MainDf is now size (23451, 1010)


Added gTurf_s15_t5.csv of size (3051, 1010) to mainDf. 
Data series completed: 6/32
MainDf is now size (26502, 1010)


Added gTurf_s15_t6.csv of size (4952, 1010) to mainDf. 
Data series completed: 7/32
MainDf is now size (31454, 1010)


Added gTurf_s15_t7.csv of size (4950, 1010) to mainDf. 
Data series completed: 8/32
MainDf is now size (36404, 1010)


Added gTurf_s15_t8.csv of size (6367, 10

In [5]:
# avoid reloading dataset when working
df = mainDf.copy(deep=True)

In [6]:
df.head(10)

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,TimeDelta1,TimeDelta1Exp2,TimeDelta11,TimeDelta11Exp2,TimeDelta21,TimeDelta21Exp2,TimeDelta31,TimeDelta31Exp2,TimeDelta41,TimeDelta41Exp2,TimeDelta51,TimeDelta51Exp2,TimeDelta61,TimeDelta61Exp2,TimeDelta71,TimeDelta71Exp2,TimeDelta81,TimeDelta81Exp2,TimeDelta91,TimeDelta91Exp2,TimeDelta101,TimeDelta101Exp2,TimeDelta111,TimeDelta111Exp2,TimeDelta121,TimeDelta121Exp2,TimeDelta131,TimeDelta131Exp2,TimeDelta141,TimeDelta141Exp2,TimeDelta151,TimeDelta151Exp2,TimeDelta161,TimeDelta161Exp2,TimeDelta171,TimeDelta171Exp2,TimeDelta181,TimeDelta181Exp2,TimeDelta191,TimeDelta191Exp2,TimeDelta201,TimeDelta201Exp2,TimeDelta211,TimeDelta211Exp2,TimeDelta221,...,OdomPosXYDelta21Exp2,OdomPosXYDelta31,OdomPosXYDelta31Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta51,OdomPosXYDelta51Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta71,OdomPosXYDelta71Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta91,OdomPosXYDelta91Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta111,OdomPosXYDelta111Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta131,OdomPosXYDelta131Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta151,OdomPosXYDelta151Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta171,OdomPosXYDelta171Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta191,OdomPosXYDelta191Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta211,OdomPosXYDelta211Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta231,OdomPosXYDelta231Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta251,OdomPosXYDelta251Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta271,OdomPosXYDelta271Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta291,OdomPosXYDelta291Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Terrain,Trial
0,0.095387,0.995440,0.148886,-0.006150,-0.004592,-0.004974,-0.104227,-0.994526,0.000000,-0.027669,-0.013835,-0.290297,-0.049081,9.154829,18.806006,0.005092,0.000026,0.065911,0.004344,0.130681,0.017078,0.187764,0.035255,0.251098,0.063050,0.312472,0.097639,0.368369,0.135695,0.425546,0.181089,0.493427,0.243470,0.557480,0.310784,0.636363,0.404958,0.716054,0.512733,0.771192,0.594737,0.834672,0.696677,0.892386,0.796352,0.955203,0.912412,1.017193,1.034682,1.074467,1.154480,1.138310,1.295749,1.201327,1.443186,1.258815,1.584615,1.315898,1.731587,1.381064,...,7.655496e-08,-0.000377,1.421181e-07,-0.000521,2.718834e-07,-0.000655,4.283823e-07,-0.000789,6.224590e-07,-0.000925,8.556460e-07,-0.001140,0.000001,-0.001399,0.000002,-0.001534,0.000002,-0.001727,0.000003,-0.001878,0.000004,-0.002115,0.000004,-0.002290,0.000005,-0.002530,0.000006,-0.002780,0.000008,-0.002953,0.000009,-0.003253,0.000011,-0.003507,0.000012,-0.003737,0.000014,-0.003969,0.000016,-0.004253,0.000018,-0.004460,0.000020,-0.004768,0.000023,-0.005013,0.000025,-0.005302,0.000028,-0.005564,0.000031,-0.005779,0.000033,-0.006097,0.000037,-0.006436,0.000041,15,MitTile,8
1,0.095366,0.995442,0.148886,-0.007478,-0.003933,-0.004679,-0.104174,-0.994536,0.008514,-0.064916,-0.013835,-0.099958,-0.174777,10.842142,18.805998,0.005561,0.000031,0.065683,0.004314,0.130839,0.017119,0.188547,0.035550,0.251223,0.063113,0.307715,0.094688,0.368386,0.135708,0.425174,0.180773,0.493495,0.243537,0.557880,0.311230,0.634416,0.402483,0.715011,0.511240,0.770069,0.593006,0.832622,0.693260,0.892173,0.795973,0.956060,0.914050,1.017388,1.035079,1.071423,1.147947,1.137589,1.294109,1.199377,1.438505,1.259750,1.586971,1.321424,1.746161,1.380945,...,7.574112e-08,-0.000375,1.406929e-07,-0.000519,2.694312e-07,-0.000652,4.247614e-07,-0.000780,6.089584e-07,-0.000921,8.484502e-07,-0.001137,0.000001,-0.001394,0.000002,-0.001529,0.000002,-0.001720,0.000003,-0.001871,0.000004,-0.002107,0.000004,-0.002279,0.000005,-0.002518,0.000006,-0.002767,0.000008,-0.002945,0.000009,-0.003228,0.000010,-0.003498,

In [7]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(145353, 1010)

### Feature engineering: elimination

In [8]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x and 'Trial' not in x) ]
df = df.drop(columns=columnsToDrop)

In [9]:
df.head(8)

,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta11,OdomOrientZDelta11Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta31,OdomOrientZDelta31Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta51,OdomOrientZDelta51Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta71,OdomOrientZDelta71Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta91,OdomOrientZDelta91Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta111,OdomOrientZDelta111Exp2,OdomOrientZDelta121,OdomOrientZDelta121Exp2,OdomOrientZDelta131,OdomOrientZDelta131Exp2,OdomOrientZDelta141,OdomOrientZDelta141Exp2,OdomOrientZDelta151,OdomOrientZDelta151Exp2,OdomOrientZDelta161,OdomOrientZDelta161Exp2,OdomOrientZDelta171,OdomOrientZDelta171Exp2,OdomOrientZDelta181,OdomOrientZDelta181Exp2,OdomOrientZDelta191,OdomOrientZDelta191Exp2,OdomOrientZDelta201,OdomOrientZDelta201Exp2,OdomOrientZDelta211,OdomOrientZDelta211Exp2,OdomOrientZDelta221,OdomOrientZDelta221Exp2,OdomOrientZDelta231,OdomOrientZDelta231Exp2,OdomOrientZDelta241,OdomOrientZDelta241Exp2,OdomOrientZDelta251,OdomOrientZDelta251Exp2,OdomOrientZDelta261,OdomOrientZDelta261Exp2,OdomOrientZDelta271,OdomOrientZDelta271Exp2,OdomOrientZDelta281,OdomOrientZDelta281Exp2,OdomOrientZDelta291,OdomOrientZDelta291Exp2,...,OdomPosXYDelta21Exp2,OdomPosXYDelta31,OdomPosXYDelta31Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta51,OdomPosXYDelta51Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta71,OdomPosXYDelta71Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta91,OdomPosXYDelta91Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta111,OdomPosXYDelta111Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta131,OdomPosXYDelta131Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta151,OdomPosXYDelta151Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta171,OdomPosXYDelta171Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta191,OdomPosXYDelta191Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta211,OdomPosXYDelta211Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta231,OdomPosXYDelta231Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta251,OdomPosXYDelta251Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta271,OdomPosXYDelta271Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta291,OdomPosXYDelta291Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Terrain,Trial
0,-0.000021,4.593532e-10,-0.000117,1.369213e-08,-0.000153,2.334641e-08,-0.000310,9.633534e-08,-0.000206,4.228962e-08,-0.000226,5.090268e-08,0.000041,1.700049e-09,0.000249,6.188848e-08,0.000260,6.770568e-08,0.000309,9.531214e-08,0.000506,2.555428e-07,0.000735,5.402217e-07,0.000994,9.888575e-07,0.001268,0.000002,0.001392,0.000002,0.001665,0.000003,0.001813,0.000003,0.001870,0.000003,0.002095,0.000004,0.002160,0.000005,0.002305,0.000005,0.002364,0.000006,0.002273,0.000005,0.002314,0.000005,0.002332,0.000005,0.002269,0.000005,0.002001,0.000004,0.001409,0.000002,0.001562,0.000002,0.001738,0.000003,...,7.655496e-08,-0.000377,1.421181e-07,-0.000521,2.718834e-07,-0.000655,4.283823e-07,-0.000789,6.224590e-07,-0.000925,8.556460e-07,-0.001140,0.000001,-0.001399,0.000002,-0.001534,0.000002,-0.001727,0.000003,-0.001878,0.000004,-0.002115,0.000004,-0.002290,0.000005,-0.002530,0.000006,-0.002780,0.000008,-0.002953,0.000009,-0.003253,0.000011,-0.003507,0.000012,-0.003737,0.000014,-0.003969,0.000016,-0.004253,0.000018,-0.004460,0.000020,-0.004768,0.000023,-0.005013,0.000025,-0.005302,0.000028,-0.005564,0.000031,-0.005779,0.000033,-0.006097,0.000037,-0.006436,0.000041,15,MitTile,8
1,-0.000021,4.593532e-10,-0.000132,1.753519e-08,-0.000181,3.267875e-08,-0.000301,9.071220e-08,-0.000227,5.162486e-08,-0.000266,7.083861e-08,-0.000018,3.331765e-10,0.000214,4.579627e-08,0.000254,6.460856e-08,0.000285,8.123373e-08,0.000447,1.998104e-07,0.000673,4.527571e-07,0.000953,9.088974e-07,0.001244,0.000002,0.001343,0.000002,0.001662,0.000003,0.001785,0.000003,0

### Speed filtering

In [10]:
df = df[df['Speed']==15]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [11]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']<9]
dfTest = df[df['Trial']>=9]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [12]:
featureCount = 1
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)
# print(fit.scores_)
print(X_test.columns.tolist()[fit.scores_.argmax()])
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

ImuAngVelXDelta21Exp2


In [13]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [14]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(123978, 1)
(123978,)
(21375, 1)
(21375,)


In [15]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.25080701754385964
